In [23]:
import torchvision.models as models
import boto3
import sagemaker
import pandas as pd 
import torch 
import re
import glob
import os
from torch.utils.data import Dataset
from torchvision.io import read_image
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn

In [3]:
model = models.resnet18(pretrained=True)

In [4]:
session = sagemaker.Session()

In [5]:
bucket_name = session.default_bucket()

In [6]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [7]:
bucket_obj = s3_resource.Bucket(bucket_name)

In [8]:
def annotation_maker(image_dir: str, output_filename: str) -> None:
    
    files = glob.glob(f'{image_dir}/*/*')
    
    classes = [int(re.findall(r'\d+', file)[0]) for file in files]
    short_files = [os.path.join(file.split('/')[-2],file.split('/')[-1]) for file in files]
    
    (pd.DataFrame({'files': short_files,
                   'classes': classes})
     .to_csv(os.path.join(image_dir,output_filename), index=False, encoding = 'utf-8', header=False))
    
    return None
    
    

In [9]:
for folder in glob.glob('dogImages/*'):
    annotation_maker(folder, 'annotation.csv')

In [11]:
classes = [int(re.findall(r'\d+',item.key)[0]) for item in bucket_obj.objects.filter(Prefix = 'dog-breed-data//train')]
keys = [item.key for item in bucket_obj.objects.filter(Prefix = 'dog-breed-data//train')]
class_names = [item.key.split('.')[1].split('/')[0] for item in bucket_obj.objects.filter(Prefix = 'dog-breed-data//train')]

In [19]:
class_name_df = pd.DataFrame({'class':classes, 'names':class_names}).drop_duplicates()

class_name_df['class'].max()

133

In [20]:
class_name_df.shape

(133, 2)

In [13]:


class dogDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [14]:



#https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.resnet50

transform_train = transforms.Compose(
        [
            transforms.RandomCrop(256, pad_if_needed = True),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.2023, 0.1994, 0.2010])
        ])

transform_test_valid = transforms.Compose(
        [
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.2023, 0.1994, 0.2010])
        ])


In [15]:
train_dataset = dogDataset(annotations_file = './dogImages/train/annotation.csv', img_dir='./dogImages/train/', transform = transform_train)
valid_dataset = dogDataset(annotations_file = './dogImages/valid/annotation.csv', img_dir='./dogImages/valid/', transform = transform_test_valid)
test_dataset = dogDataset(annotations_file = './dogImages/test/annotation.csv', img_dir = './dogImages/valid/', transform = transform_test_valid)

In [16]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 30, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = 30, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 30, shuffle = True)


In [ ]:
for images in valid_loader:
    for image in images:
        model(image)

In [138]:
tensor = read_image('./dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg')

In [139]:
tensor.max()

tensor(255, dtype=torch.uint8)

In [9]:
example = list(valid_loader)[0][0]

[2022-07-23 05:23:40.797 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:903 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-07-23 05:23:41.066 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:903 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [11]:
example_response = model(example)

In [12]:
example_response.shape

torch.Size([30, 1000])

In [1]:
print('duck')

duck


In [26]:
def net():
    model = models.resnet18(pretrained=True)
    
    #set the gradients to false - do not train the gradients of resnet
    for parameter in model.parameters():
        parameter.requires_grad = False
        
    #fc means something like final fully connected layer - we replace this with the characteristics we want.
    pretrained_output_parameters = model.fc.in_features
    model.fc = nn.Sequential(
                             nn.Linear(pretrained_output_parameters,500),
                             nn.ReLU(),
                             nn.Linear(500, 133)
    )

    return model
    
    

In [28]:
model = net()

In [29]:
import os


In [30]:
import wget

ModuleNotFoundError: No module named 'wget'

In [31]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9673 sha256=9d9afd9dd6e7c783c10113cc36d8545ccc53db2a878af0b174110975f4d7e371
  Stored in directory: /root/.cache/pip/wheels/90/1d/93/c863ee832230df5cfc25ca497b3e88e0ee3ea9e44adc46ac62
Successfully built wget


In [32]:
from torch.utils.data import datasets

In [3]:
from torchvision.datasets import ImageFolder

In [2]:
train_data = ImageFolder('./dogImages/train/', transform_train)

NameError: name 'ImageFolder' is not defined

In [4]:
import os

In [8]:
os.mkdirs('path/to/object')

AttributeError: module 'os' has no attribute 'mkdirs'

In [4]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

In [5]:
transform_train = transforms.Compose(
                                        [
                                            transforms.RandomCrop(256, pad_if_needed = True),
                                            transforms.RandomHorizontalFlip(p=0.5),
                                            transforms.ToTensor(),
                                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.2023, 0.1994, 0.2010])
                                        ])

In [12]:
train_data = ImageFolder(root = './dogImages/train',
                         transform = transform_train,
                        )




In [7]:
from torch import optim

In [8]:
optim.Adagrad

torch.optim.adagrad.Adagrad

In [13]:
train_data

Dataset ImageFolder
    Number of datapoints: 6680
    Root location: ./dogImages/train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(256, 256), padding=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.2023, 0.1994, 0.201])
           )

In [15]:
max(train_data.targets)

133

In [16]:
import boto3

s3_res = boto3.resource('s3')
s3_client = boto3.client('s3')

In [17]:
import sagemaker

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [18]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'dog-breed-data'

In [20]:
bucket

'sagemaker-us-east-1-532709353901'

In [21]:
import os

In [22]:
os.path.exists('./dogImages')

True

In [19]:
sagemaker_session.download_data(path = './backupDogImages',bucket=bucket, key_prefix= 'dog-breed-data/')

KeyboardInterrupt: 

In [23]:
import torch

In [26]:
torch.cuda.get_device_name('cpu')

AssertionError: Torch not compiled with CUDA enabled

In [24]:
torch.cuda.device

torch.cuda.device

In [27]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [28]:
from torch import nn

In [31]:
model = nn.Sequential(nn.Linear(40,10))

In [34]:
model = model.to(device)